***This is the template for creating a readme lesson for Data Science. Please make sure to change the title to the appropriate topic, add an introduction, objectives (which should be taken from the lesson's outlined SWBATS), the lesson's content, and summary.***

> Note: If possible, structure the lesson's content according to the listed objectives. This helps to give a flow and framework for the material being presented to students.

# LESSON TITLE

## Introduction
Introduction goes here

## Objectives
You will be able to:
* SWBATS 
* GO 
* HERE

## Objective 1 Title

In [ ]:
# Objective 1 content

## Objective 2 Title

In [ ]:
## Objective 2 content

## Objective 3 Title

In [ ]:
## Objective 3 content

## Summary
Summary goes here